In [2]:
!pip install librosa

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23141 sha256=f28d492fac1861274569e83431b749064d5603d69205a63b64b3c51e2b91f4c6
  Stored in directory: c:\users\iyeng\appdata\local\pip\cache\wheels\49\5a\e4\df590783499a992a88de6c0898991d1167453a3196d0d1eeb7
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320718 sha256=c8f37fc82d40cc7bdd81ead6ca70301bd5118000c82862386d4a898205cf08ba
  Stored in directory: c:\users\iyeng\appdata\local\pip\cache\wheels\6f\d1\5d\f13da53b1dcbc2624ff548456c9ffb526c914f53c12c318bb4
Successfully built audioread resampy


In [3]:
import pandas as pd
import numpy as np
import librosa
from librosa import feature
from glob import glob
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
min_max_scaler = preprocessing.MinMaxScaler()


In [86]:
# rootdir = '/Users/abhishekvaidyanathan/Desktop/NNDL-project/audio-files'
rootdir = '/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/GroupProject/audio-files'
audio_files = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        audio_files.append(os.path.join(subdir, file))


In [87]:
len(audio_files)

1440

In [88]:
audio_files[1][-33:-25]

'Actor_01'

In [89]:
dict_actors = {}
for audio_file in audio_files:
    try :
        if (len(dict_actors[audio_file[-33:-25]])>0):
            dict_actors[audio_file[-33:-25]].append(audio_file)
    except:
        dict_actors[audio_file[-33:-25]] = []
        dict_actors[audio_file[-33:-25]].append(audio_file)

In [90]:
data = pd.DataFrame(columns = ['Actor','Modality','Vocal_channel','Emotion','Emotional_intensity','Statement','Repetion','Gender','Audio_file'])

In [91]:
def get_gender(value):
    if(int(value)%2==0):
        return 1
    else :
        return 0

for keys in dict_actors:
    for files in dict_actors[keys]:
        data.loc[len(data)] = [keys,int(files[-24:-22]),int(files[-21:-19]),int(files[-18:-16]),int(files[-15:-13]),int(files[-12:-10]),int(files[-9:-7]),get_gender(files[-6:-4]),files]

In [92]:
data.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender,Audio_file
0,Actor_01,3,1,1,1,1,1,0,/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/Grou...
1,Actor_01,3,1,1,1,1,2,0,/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/Grou...
2,Actor_01,3,1,1,1,2,1,0,/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/Grou...
3,Actor_01,3,1,1,1,2,2,0,/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/Grou...
4,Actor_01,3,1,2,1,1,1,0,/Users/iyeng/Desktop/NTU/NTU Sem 5/CZ4042/Grou...


## can change the below code cell to include different set of features. 

#### right now uses mean values, can change to using raw values for each of the features. 

In [38]:
# y, sr = librosa.load(librosa.util.example_audio_file())
# temp_func = feature.spectral_bandwidth
# temp_res = temp_func(y=y, sr=sr)
# print(temp_res)
# print(temp_res.shape)
# print(np.mean(temp_res))

[[2803.66001659 1379.6672431  1562.99924373 ... 2549.85781492
  2456.23949636 2515.19654634]]
(1, 2647)
1364.8838771312614


In [41]:
# to add:
# mfcc - use n_mfcc=13 and take mean along axis 1 [13 features]
# chroma_stft - take mean along axis 1 [12 features]
# librosa.onset.onset_strength(y=y, sr=sr) - take direct mean
# zero_crossing_rate - take direct mean
# spectral_rolloff - direct mean
# librosa.piptrack - returns pitch and magnitude, take direct means of both
# melspectrogram - take direct mean
# spectral_contrast - use axis=1 [7 features]
# tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0) [6 features]
# rms - take direct mean
# spectral_centroid - take direct mean
# spectral_bandwidth - take direct mean


def get_feature_vector(y, sr):
    feature_vector = []
    
    # multi-dim features
    feature_vector.extend(np.mean(feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1))
    feature_vector.extend(np.mean(feature.chroma_stft(y=y, sr=sr), axis=1))
    feature_vector.extend(np.mean(feature.spectral_contrast(y=y, sr=sr), axis=1))
    feature_vector.extend(np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr),axis=1))
    
    # single-dim features with special requirements
    feature_vector.append(np.mean(feature.rms(y=y)))
    feature_vector.append(np.mean(feature.zero_crossing_rate(y=y)))
    feature_vector.extend([np.mean(x) for x in librosa.piptrack(y=y, sr=sr)])
    
    # single-dim features
    feat_list = [
        librosa.onset.onset_strength,
        feature.spectral_rolloff,
        feature.melspectrogram,
        feature.spectral_centroid,
        feature.spectral_bandwidth
    ]
    
    for temp_func in feat_list:
        feature_vector.append(np.mean(temp_func(y=y, sr=sr)))
    
    return feature_vector

In [93]:
audio_features = []
for i in range(data.shape[0]):
   y , sr = librosa.load(data.iloc[i]['Audio_file'],sr=None)
   feature_vector = get_feature_vector(y, sr)
   audio_features.append(feature_vector) 

In [94]:
data_features = data.copy()

In [95]:
data_features = data_features.drop("Audio_file",axis=1)
data_features.head()

,Actor,Modality,Vocal_channel,Emotion,Emotional_intensity,Statement,Repetion,Gender
0,Actor_01,3,1,1,1,1,1,0
1,Actor_01,3,1,1,1,1,2,0
2,Actor_01,3,1,1,1,2,1,0
3,Actor_01,3,1,1,1,2,2,0
4,Actor_01,3,1,2,1,1,1,0


In [96]:
data_features['librosa'] = audio_features

In [97]:
features = pd.DataFrame(data_features['librosa'].values.tolist())

In [98]:
statement1 = 'Kids are talking by the door'
statement2 = 'Dogs are sitting by the door'

In [99]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,-726.217224,68.541420,3.293397,12.205300,5.510278,13.667408,-2.983829,3.098029,-3.310813,-1.564384,...,0.012057,0.002120,0.050476,37.457439,0.002697,0.891978,13286.038306,0.002952,7416.379545,5551.324979
1,-719.127808,70.202240,1.169071,13.123216,7.837617,14.411950,-4.110705,4.469619,-3.538730,-3.657982,...,0.004175,0.002258,0.052904,40.032833,0.002662,0.992758,13191.718251,0.003416,7135.753114,5653.712371
2,-714.994934,69.690376,3.925557,11.925324,6.423343,11.014113,-2.874456,4.514386,-4.470305,-2.665093,...,0.016821,0.002707,0.046627,37.690022,0.003359,0.860653,13280.282980,0.004721,7240.619346,5640.892215
3,-710.959839,67.579193,5.783356,13.227695,6.194669,12.640195,-1.662046,5.663977,-4.953693,-3.484669,...,0.011100,0.002521,0.053835,39.524185,0.003164,0.918893,13273.018037,0.004341,7009.490125,5802.602446
4,-759.917847,75.788948,6.028997,14.562723,6.459432,14.636641,-2.999552,4.625813,-5.195350,-0.702961,...,0.002851,0.001579,0.045929,38.651924,0.001842,0.988106,12649.614081,0.001624,6997.114097,5518.781643


In [100]:
features.shape

(1440, 47)

In [101]:
labels = data['Emotion']

In [102]:
actor_labels = pd.Series([int(x[-2:]) for x in data['Actor']], name='Actor')

In [103]:
features_values = features.values

In [104]:
features_scaled_values = min_max_scaler.fit_transform(features_values)
features_normalised = pd.DataFrame(features_scaled_values)

In [105]:
features_normalised.head()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.272337,0.515438,0.739652,0.549830,0.645790,0.640857,0.672820,0.601139,0.657816,0.688467,...,0.571816,0.020323,0.161635,0.639285,0.020555,0.125028,0.860550,0.000705,0.952777,0.774063
1,0.285469,0.532704,0.711228,0.566807,0.698543,0.659506,0.644720,0.642219,0.649567,0.615905,...,0.490164,0.021892,0.178884,0.686954,0.020258,0.232647,0.850756,0.000818,0.897211,0.802389
2,0.293124,0.527383,0.748110,0.544651,0.666486,0.574399,0.675547,0.643559,0.615851,0.650318,...,0.621171,0.026986,0.134300,0.643590,0.026162,0.091577,0.859952,0.001138,0.917975,0.798843
3,0.300599,0.505434,0.772967,0.568739,0.661303,0.615128,0.705780,0.677990,0.598355,0.621912,...,0.561902,0.024872,0.185497,0.677539,0.024511,0.153769,0.859198,0.001045,0.872211,0.843582
4,0.209913,0.590785,0.776254,0.593432,0.667304,0.665134,0.672428,0.646897,0.589609,0.718323,...,0.476445,0.014187,0.129345,0.661394,0.013311,0.227678,0.794469,0.000379,0.869760,0.765059


In [60]:
X_train, X_test, y_train, y_test = train_test_split(features_normalised, labels, test_size=0.30, random_state=42)

In [61]:
X_train.to_csv('SER_data/X_train.csv', index=False)
X_test.to_csv('SER_data/X_test.csv', index=False)
y_train.to_csv('SER_data/y_train.csv', index=False)
y_test.to_csv('SER_data/y_test.csv', index=False)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(features_normalised, actor_labels, test_size=0.30, random_state=42)

In [73]:
X_train.to_csv('Speaker_Classification_data/X_train.csv', index=False)
X_test.to_csv('Speaker_Classification_data/X_test.csv', index=False)
y_train.to_csv('Speaker_Classification_data/y_train.csv', index=False)
y_test.to_csv('Speaker_Classification_data/y_test.csv', index=False)

In [106]:
total_pairs = []
for i in range(len(features_normalised)-1):
    for j in range(i+1, len(features_normalised)):
        f1 = features_normalised.loc[i, :].values.tolist()
        f2 = features_normalised.loc[j, :].values.tolist()
        if actor_labels[i] == actor_labels[j]:
            l = 1
        else:
            l = 0
        total_pairs.append([f1, f2, l])
pairs_df = pd.DataFrame(total_pairs, columns=['inputA', 'inputB', 'label'])

In [107]:
pairs_df.head()

,inputA,inputB,label
0,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.2854689531660164, 0.5327042826670929, 0.711...",1
1,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.2931243461802795, 0.5273828194015168, 0.748...",1
2,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.3005986227234241, 0.5054344306390837, 0.772...",1
3,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.20991285946164528, 0.5907850984592853, 0.77...",1
4,"[0.27233710455813176, 0.5154379820620423, 0.73...","[0.25605640704038835, 0.6251354289835376, 0.80...",1


In [109]:
pairs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036080 entries, 0 to 1036079
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   inputA  1036080 non-null  object
 1   inputB  1036080 non-null  object
 2   label   1036080 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 23.7+ MB


In [110]:
matched_df = pairs_df[pairs_df['label']==1].copy()
mismatched_df = pairs_df[pairs_df['label']==0].copy()

In [111]:
matched_df.describe()

,label
count,42480.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [112]:
mismatched_df.describe()

,label
count,993600.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [113]:
# shuffling
matched_df = matched_df.sample(frac=1).reset_index(drop=True)
mismatched_df = mismatched_df.sample(frac=1).reset_index(drop=True)

In [115]:
downsampled_df = pd.concat([matched_df, mismatched_df[:len(matched_df)].copy()], ignore_index=True)

In [116]:
downsampled_df.describe()

,label
count,84960.000000
mean,0.500000
std,0.500003
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [117]:
# shuffling and taking train-test split

downsampled_df = downsampled_df.sample(frac=1).reset_index(drop=True)
train_size = int(0.7*len(downsampled_df))
train_df = downsampled_df[:train_size].copy()
test_df = downsampled_df[train_size+1:].copy()

In [122]:
test_df.reset_index(drop=True, inplace=True)

In [118]:
train_df.head()

,inputA,inputB,label
0,"[0.24985932948240364, 0.6350656914100974, 0.82...","[0.7820350888855649, 0.3198341320895355, 0.303...",1
1,"[0.3447592728071933, 0.7605236831924497, 0.789...","[0.6068565133586963, 0.4892604057585005, 0.647...",1
2,"[0.5835598702428237, 0.2923623783944991, 0.494...","[0.7092419823457793, 0.3953920733270686, 0.584...",0
3,"[0.6494945188525626, 0.33861761010055624, 0.29...","[0.412796658774194, 0.6307090412962476, 0.7552...",0
4,"[0.5530646953235432, 0.46914860434629924, 0.48...","[0.605411651622807, 0.5611188390198073, 0.5267...",1


In [123]:
test_df.head()

,inputA,inputB,label
0,"[0.7599851760351478, 0.27829568977021635, 0.39...","[0.3881078382392493, 0.6320058758104472, 0.777...",1
1,"[0.23891840228138905, 0.19716560358624316, 0.7...","[0.1339817574334914, 0.3268776091606487, 0.788...",1
2,"[0.75814829097616, 0.38292398110746095, 0.6330...","[0.3696297750956259, 0.5905143892117646, 0.885...",0
3,"[0.2607263177495158, 0.6446019065053701, 0.742...","[0.729151566561377, 0.1314536503611208, 0.2343...",0
4,"[0.7674167172311745, 0.26044605158506295, 0.08...","[0.8528030713373913, 0.2308533587518186, 0.265...",1


In [120]:
train_df.to_csv('Speaker_Classification_data/train_data.csv', index=False)
test_df.to_csv('Speaker_Classification_data/test_data.csv', index=False)

In [124]:
train_df.to_pickle('Speaker_Classification_data/train.df')
test_df.to_pickle('Speaker_Classification_data/test.df')